<a href="https://colab.research.google.com/github/alitourani/Iranis-dataset/blob/master/examples/colab/recommender_get_grid_vmf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **🍿 Popcorn Framework in Google Colab**
### **Recommender - Prepare Grid Computing (VMF)**

🎬 Popcorn Framework: [link](https://github.com/RecSys-lab/Popcorn)

## **[Step 1] Clone Popcorn Movie Recommender Tool**

Clone the framework into your `GDrive` and prepare it for experiments.

⚠️ You might see a *"Restart Session"* warning during the first run in Google Colab due to library version mismatches. This is expected! Accept the restart, re-run this cell, and continue!

In [1]:
# Clone the repo
!git clone https://github.com/RecSys-lab/Popcorn.git

# Install the required library
%cd Popcorn
!pip install -e .

# Add the repository to the Python path
import sys
sys.path.append('/content/Popcorn')

# Go back to the root
%cd ..

fatal: destination path 'Popcorn' already exists and is not an empty directory.
/content/Popcorn
Obtaining file:///content/Popcorn
  Preparing metadata (setup.py) ... done
  Attempting uninstall: Popcorn
    Found existing installation: Popcorn 1.6.0
    Uninstalling Popcorn-1.6.0:
      Successfully uninstalled Popcorn-1.6.0
  Running setup.py develop for Popcorn
/content


## 🚀 **[Step 2] Use the Framework**

### *1. Load Configurations and Imports*

In [2]:
import os
import json
import pandas as pd
from popcorn.utils import readConfigs

# Start the Framework
print("Welcome to 'Popcorn' 🍿! Starting the framework for your movie recommendation ...\n")

# Read the configuration file
configs = readConfigs("Popcorn/popcorn/config/config.yml")
# If properly read, print the configurations
if not configs:
    print("Error reading the configuration file!")

Welcome to 'Popcorn' 🍿! Starting the framework for your movie recommendation ...

- Reading the framework's configuration file ...
- Configuration file loaded successfully!


In [3]:
# Override Configurations
configs["datasets"]["unimodal"]["movielens"]["version"] = "1m"        # '100k' | '1m' | '25m'
configs["datasets"]["multimodal"]["mmtf"]["audio_variant"] = "ivec"   # 'blf' | 'ivec'
configs["datasets"]["multimodal"]["mmtf"]["visual_variant"] = "cnn"   # 'avf' | 'cnn'
configs["modalities"]["fusion_methods"]["selected"] = ["concat"]      # 'concat' | 'cca' | 'pca'
configs["modalities"]["selected"] = ["audio_mmtf", "visual_mmtf", "text_rag_plus"]

configs["setup"]["use_gpu"] = False  # Disable GPU for grid search
configs["setup"]["model_choice"] = "vmf"  # Model choice for grid search
configs["setup"]["use_parallel"] = True  # Enable parallel processing for grid search
configs["setup"]["is_fast_prototype"] = True  # Enable fast prototype mode for grid search

### *2. Assemble Modalities (MMTF-14K and RAG-Plus)*

In [4]:
from popcorn.recommenders.assembler import assembleModality

trainDF, testDF, trainSet, modalitiesDict, genreDict = assembleModality(configs)
if trainDF is None or testDF is None or trainSet is None:
  print("- Error in assembling modalities! Exiting ...")

print("- Modalities assembled successfully!")
print(f"- Training set size: {trainDF.shape}, Testing set size: {testDF.shape}")
print(f"- Available modalities: {list(modalitiesDict.keys())}")


- Downloading the MovieLens-1m dataset ...
- Creating the download path 'E:/Datasets/MovieLens/ml-1m' ...
- Fetching data from 'https://files.grouplens.org/datasets/movielens/ml-1m.zip' ...
- Download completed and the dataset is saved as a 'zip' file!
- Extracting the dataset files inside 'E:/Datasets/MovieLens/ml-1m' ...
- Dataset extracted to 'E:/Datasets/MovieLens/ml-1m' successfully!
- Removing the zip file 'E:/Datasets/MovieLens/ml-1m/ml-1m.zip' ...
- Zip file removed successfully!

- Loading 'MovieLens-1m' data from 'E:/Datasets/MovieLens/ml-1m/ml-1m' ...
- Items (movies) have been loaded. Number of rows: 3,883
- Users have been loaded. Number of rows: 6,040
- Ratings have been loaded. Number of rows: 1,000,209

- Preparing to save the genres DataFrame in 'outputs' ...
- Genres DataFrame saved to 'outputs/item_genre_ml-1m.csv'!
- Genre dictionary loaded: 
{'1': ['Animation', "Children's", 'Comedy'], '2': ['Adventure', "Children's", 'Fantasy'], '3': ['Comedy', 'Romance']}
- Appl

In [5]:
trainDF.head(5)

,user_id,item_id,rating,timestamp
0,2993,2606,1,970740610
1,5952,265,4,957144461
2,2664,2378,3,973455388
3,2777,1892,4,973051349
4,4360,1911,2,965189832


In [6]:
testDF.head(5)

,user_id,item_id,rating,timestamp
0,922,1721,4,975200962
1,5744,1193,5,960857360
2,1597,3039,4,1035294422
3,1880,1302,4,975379470
4,1938,1373,2,974698240


### *3. Run GridSearch*

In [7]:
from popcorn.optimizers.grid_search import gridSearch

finalModels = gridSearch(configs, trainDF, trainSet, modalitiesDict)
print("- Grid search completed successfully!")
print(f"- Final models after HPO: {list(finalModels.keys())}")

- Preparing GridSearch procedure ...
- Training and validation sets prepared!
- Preparing parameter grid for optimizers (prototype='True', epochs=10) ...
- Starting HPO ...
- Starting GridSearch procedure (seed: 42, useGPU: False, parallelHPO: True)...
-- HPO 'VMF' in scenario 'visual' with 3 param sets...
-- Fitting '{'k': 32, 'learning_rate': 0.01, 'n_epochs': 1}' to get 0.0000 ...
-- Fitting '{'k': 64, 'learning_rate': 0.01, 'n_epochs': 1}' to get 0.3377 ...
-- Fitting '{'k': 128, 'learning_rate': 0.01, 'n_epochs': 1}' to get 0.0000 ...
-- Found the best case: 'VMF' in 'visual', item '{'k': 64, 'learning_rate': 0.01, 'n_epochs': 1}' (0.3377), done in 96.1s.
- Starting GridSearch procedure (seed: 42, useGPU: False, parallelHPO: True)...
-- HPO 'VMF' in scenario 'audio' with 3 param sets...
-- Fitting '{'k': 32, 'learning_rate': 0.01, 'n_epochs': 1}' to get 0.2281 ...
-- Fitting '{'k': 64, 'learning_rate': 0.01, 'n_epochs': 1}' to get 0.0000 ...
-- Fitting '{'k': 128, 'learning_rate':